## Predicting Fraud Medical Claims
**Author**: Andrew Kruhko

### Scope

The scope of this notebook is to provide instructions on how to run a fraud medical claims use case with Python and DataRobot. The data is ingested through a database so you need to manipulate the code accordingly to fit your own needs but this serves as a prime example of how you can connect DataRobot to your own source systems. 

### Requirements

- Python version 3.7.3
-  DataRobot API version 2.19.0. 
Small adjustments might be needed depending on the Python version and DataRobot API version you are using.

Full documentation of the Python package can be found here: https://datarobot-public-api-client.readthedocs-hosted.com

#### Import Libraries
Importing libraries that will be used in the project.

In [1]:
import pandas as pd
import numpy as np
import datarobot as dr
import yaml
import time
import json
import requests

from datetime import datetime

pd.options.display.max_columns = 100

#### Connecting to DataRobot
Using <code>dr.Client</code> command alongside a yaml configuration file, we can connect to DataRobot. Alternatively, we could pass the <code>token</code> and <code>endpoint</code> variables to achieve the same result.

In [3]:
dr.Client(config_path="../drconfig.yaml")

#### Define some variables for later...
Here we are defining some variables that will be used later in the analysis

Yaml file description and required tags: 
- token: a DR API key
- username: a login to DR
- db_address: an IP or a server name
- db_name: a database to connect with
- db_user: a dadabase login
- db_pass: a database password
- pred_serv_id: a DR prediction server id

In [ ]:
with open("../drconfig.yaml", 'r') as stream:
    creds = yaml.safe_load(stream)
token = creds['token']

project_name = 'Medical_Insurance_Fraud'
target = 'fraud'
metric = 'LogLoss'

base_url = 'YOUR_DATAROBOT_HOSTNAME'

### Overview of Next Steps

1. Create data sources
    1. Find Microsoft SQL Server jdbc driver among predefined
    2. Create a data store
    3. Create a data source

2. Create and run projects
    1. Supervised models
    2. Anomaly detection models
    3. Supervised with anomaly detection features
3. Compare results
4. Deployment and scoring

#### 1. Create data sources

In [4]:
def dr_rest_call(url, req_func, payload=None):
    headers = {'Authorization': f'Token {token}',
               'Content-Type': 'application/json;charset=UTF-8'}
    return req_func(f'{base_url}{url}', headers=headers, json=payload)

#### 1.A Find Microsoft SQL Server jdbc driver among predefined

In [1]:
drivers = dr_rest_call('externalDataDrivers', requests.get)

drivers = drivers.json()
drivers_sql_serv = []
for driver in drivers['data']:
    if 'Microsoft SQL Server' in driver['canonicalName']:
        drivers_sql_serv.append([driver['canonicalName'], driver['version'], driver['id']])
drivers_sql_serv

In [2]:
# Use the latest available driver
driver_sql_serv_id = drivers_sql_serv[-1][-1]
driver_sql_serv_id

#### 1.B Create a data store

In [7]:
data = {'type': 'jdbc', 
        'canonicalName': 'sql_server_test', 
        'params': {'driverId': driver_sql_serv_id, 
                   'jdbcFields': 
                   [{'name': 'address', 'value': creds['db_address']},
                    {'name': 'databaseName', 'value': creds['db_name']}]
                  }}

data_store_resp = dr_rest_call('externalDataStores', requests.post, payload=data)

In [8]:
# get data store id
data_store = data_store_resp.json()
data_store_id = data_store['id']
data_store_id

'YOUR_DATA_STORE_ID'

#### 1.C Create a data source

In [9]:
# create a data source based on query 
ad_query = """
select *
from drdemodb1.cfds_demo.Fraud_Medical_Insurance_Fraud
order by npi
"""
data = {'type': 'jdbc', 
        'canonicalName': 'Medical_Insurance_Fraud_query', 
        'params': {'dataStoreId': data_store_id, 
                   'query': ad_query}}

data_src_query_resp = dr_rest_call('externalDataSources', requests.post, payload=data)

In [4]:
data_src_query = data_src_query_resp.json()
data_src_query_id = data_src_query['id']
data_src_query_id

### 2. Create and run projects
We will be creating multiple functions that will be used later on

In [11]:
def wait_for_proj_id(resp):
    """
    wait for the project creation
    return the project id
    """
    while True:
        resp_stat = requests.get(resp.headers['Location'], 
                                 headers={'Authorization': f'Token {token}', 
                                          'Content-Type': 'application/json;charset=UTF-8'})
        resp_stat = resp_stat.json()

        if resp_stat.get('id') is None:
            time.sleep(10)
        else:
            proj_id = resp_stat.get('id')
            break
        
    return proj_id

In [12]:
def get_model_score(mod, metric):
    res = {}
    res['model_number'] = mod.model_number
    res['model_type'] = mod.model_type
    res['model_category'] = mod.model_category
    res['model'] = mod
    res['sample_pct'] = mod.sample_pct
    
    res['metric_v'] = mod.metrics.get(metric, {}).get('validation')
    res['metric_cv'] = mod.metrics.get(metric, {}).get('crossValidation')
    
    return res

def get_model_scores(proj, metric=None, ascending=True):
    """
    iterate trough the project models and get their performance metric
    """
    if metric is None:
        metric = proj.metric        
    df = pd.DataFrame([get_model_score(m, metric) for m in proj.get_models(with_metric=metric)])
    return df.sort_values(['metric_cv', 'metric_v'], ascending=ascending, na_position='last')

In [13]:
def get_train_preds(mod):
    """
    request and/or retrieve training predictions for a given model
    """
    try:
        # request training predictions and get job ids
        pred_job = mod.request_training_predictions(dr.enums.DATA_SUBSET.ALL)
        preds = pred_job.get_result_when_complete().get_all_as_dataframe()
        return preds
    except:
        # retrieve training predictions if they were already requested
        train_preds = dr.TrainingPredictions.list(mod.project_id)
        for train_pred in train_preds:
            if train_pred.model_id == mod.id and train_pred.data_subset == 'all':
                preds = dr.TrainingPredictions.get(mod.project_id, train_pred.prediction_id).get_all_as_dataframe()
                return preds

In [14]:
def prep_ad_preds(mods):
    """
    preprocess training predictions from anomaly detection models
    mods - a list of aomaly detection models
    """
    preds = get_train_preds(mods[0])
    preds.set_index('row_id', inplace=True)
    preds = preds[['partition_id', 'class_1.0']].copy()
    preds.rename(columns={'class_1.0': f'{mods[0].model_type}_prediction'}, inplace=True)
    for mod in mods[1:]:
        preds_tmp = get_train_preds(mod)
        preds_tmp.set_index('row_id', inplace=True)
        preds = preds.merge(preds_tmp[['class_1.0']], left_index=True, right_index=True)
        preds.rename(columns={'class_1.0': f'{mod.model_type}_prediction'}, inplace=True)
        
    preds['partition_id'] = preds.partition_id.replace('Holdout', '5.0').astype(float).astype(int)
    return preds

#### 2.A Supervised Models

In [15]:
# create a project based on the data source
data = {'projectName': f'{project_name}_clf',
        'dataSourceId': data_src_query_id, 
        'user': creds['db_user'],
        'password': creds['db_pass']}

project_clf_resp = dr_rest_call('projects', requests.post, payload=data)

project_clf_id = wait_for_proj_id(project_clf_resp)

In [16]:
# find the project trough python API 
projects = dr.Project.list()
project_clf = [pr for pr in projects if pr.id == project_clf_id][0]

In [17]:
# set target and run autopilot
project_clf.set_target(target=target,
                       mode=dr.enums.AUTOPILOT_MODE.FULL_AUTO,
                       metric=metric,
                       worker_count=-1)

Project(Medical_Insurance_Fraud_clf)

In [ ]:
project_clf.wait_for_autopilot()

#### 2.B Anomaly detection models 

In [19]:
# select anomaly detection blueprints
blueprints = project_clf.get_blueprints()
blueprints = [bp for bp in blueprints if 'anomaly' in  bp.model_type.lower()]
print(len(blueprints))

8


In [20]:
# unlock holdout
project_clf.unlock_holdout()

# train anomaly detection models
print(str(datetime.now()), 'start')
model_job_ids = []
for bp in blueprints:
    model_job_ids.append(project_clf.train(bp, sample_pct=100))
    
model_jobs = []
for i in model_job_ids:
    model_jobs.append(dr.ModelJob.get(project_clf_id, i))
    
for mj in model_jobs:
    mj.wait_for_completion()
print(str(datetime.now()), 'done')

2020-04-11 17:30:31.159688 start
2020-04-11 17:34:36.464598 done


In [21]:
# get the leaderboard
model_scores = get_model_scores(project_clf, metric='AUC', ascending=False)
model_scores['is_ad'] = model_scores.model_type.apply(lambda x: 'anomaly' in x.lower())

In [22]:
# blenders show slightly better results
model_scores.head(10)

,model_number,model_type,model_category,model,sample_pct,metric_v,metric_cv,is_ad
2,124,ENET Blender,blend,Model('ENET Blender'),64.00393,0.74259,0.773014,False
10,120,Light Gradient Boosting on ElasticNet Predicti...,model,Model('Light Gradient Boosting on ElasticNet P...,100.00000,0.73000,0.772780,False
7,123,Advanced AVG Blender,blend,Model('Advanced AVG Blender'),64.00393,0.73511,0.768186,False
1,122,AVG Blender,blend,Model('AVG Blender'),64.00393,0.74596,0.767220,False
0,125,ENET Blender,blend,Model('ENET Blender'),64.00393,0.74456,0.767090,False
5,69,Light Gradient Boosting on ElasticNet Predicti...,model,Model('Light Gradient Boosting on ElasticNet P...,64.00393,0.74497,0.766502,False
4,111,Light Gradient Boosting on ElasticNet Predicti...,model,Model('Light Gradient Boosting on ElasticNet P...,64.00393,0.74298,0.763250,False
11,66,Nystroem Kernel SVM Classifier,model,Model('Nystroem Kernel SVM Classifier'),64.00393,0.72758,0.763120,False
3,68,Elastic-Net Classifier (L2 / Binomial Deviance),model,Model('Elastic-Net Classifier (L2 / Binomial D...,64.00393,0.73225,0.759464,False
8,118,Light Gradient Boosting on ElasticNet Predicti...,model,Model('Light Gradient Boosting on ElasticNet P...,80.00491,0.73110,0.752654,False


In [23]:
# anomaly detection results aren't good but let's try to use them as features for supervised models
model_scores[model_scores.is_ad].head(10)

,model_number,model_type,model_category,model,sample_pct,metric_v,metric_cv,is_ad
80,136,One-Class SVM Anomaly Detection,model,Model('One-Class SVM Anomaly Detection'),100.00000,0.57379,0.565596,True
78,131,Anomaly Detection with Supervised Learning (XGB),model,Model('Anomaly Detection with Supervised Learn...,100.00000,0.53977,0.563292,True
66,138,Local Outlier Factor Anomaly Detection,model,Model('Local Outlier Factor Anomaly Detection'),100.00000,0.56287,0.552154,True
72,137,Anomaly Detection Blender,blend,Model('Anomaly Detection Blender'),100.00000,0.48417,0.518688,True
77,135,Double Median Absolute Deviation Anomaly Detec...,model,Model('Double Median Absolute Deviation Anomal...,100.00000,0.47252,0.516750,True
75,133,Isolation Forest Anomaly Detection,model,Model('Isolation Forest Anomaly Detection'),100.00000,0.49861,0.498472,True
79,132,Mahalanobis Distance Ranked Anomaly Detection ...,model,Model('Mahalanobis Distance Ranked Anomaly Det...,100.00000,0.50817,0.496544,True
73,134,Anomaly Detection with Supervised Learning (XGB),model,Model('Anomaly Detection with Supervised Learn...,100.00000,0.47885,0.487768,True
76,43,Isolation Forest Anomaly Detection,model,Model('Isolation Forest Anomaly Detection'),16.00099,0.48111,NaN,True


#### 2.C Supervised with anomaly detection features

In [24]:
# run anomaly detection blueprints on 64% of data to avoid the target leakage
print(str(datetime.now()), 'start')
model_job_ids = []
for bp in blueprints:
    model_job_ids.append(project_clf.train(bp, sample_pct=64.00393, scoring_type=dr.enums.SCORING_TYPE.cross_validation))
    
model_jobs = []
for i in model_job_ids:
    model_jobs.append(dr.ModelJob.get(project_clf_id, i))
    
for mj in model_jobs:
    mj.wait_for_completion()
print(str(datetime.now()), 'done')

2020-04-11 17:36:23.304310 start
2020-04-11 17:40:21.983410 done


In [25]:
# get the leaderboard 
model_scores = get_model_scores(project_clf, metric='AUC', ascending=False)
model_scores['is_ad'] = model_scores.model_type.apply(lambda x: 'anomaly' in x.lower())

In [26]:
model_scores[model_scores.is_ad & (model_scores.sample_pct < 100)].head()

,model_number,model_type,model_category,model,sample_pct,metric_v,metric_cv,is_ad
87,170,One-Class SVM Anomaly Detection,model,Model('One-Class SVM Anomaly Detection'),64.00393,0.56068,0.583700,True
64,182,Local Outlier Factor Anomaly Detection,model,Model('Local Outlier Factor Anomaly Detection'),64.00393,0.54057,0.550668,True
83,140,Anomaly Detection with Supervised Learning (XGB),model,Model('Anomaly Detection with Supervised Learn...,64.00393,0.51358,0.549402,True
75,176,Anomaly Detection Blender,blend,Model('Anomaly Detection Blender'),64.00393,0.49649,0.521904,True
81,164,Double Median Absolute Deviation Anomaly Detec...,model,Model('Double Median Absolute Deviation Anomal...,64.00393,0.49306,0.518938,True


In [27]:
# select top n models
models_n = 3
models = model_scores[model_scores.is_ad & (model_scores.sample_pct < 100)].head(models_n).model.values.tolist()
models

[Model('One-Class SVM Anomaly Detection'),
 Model('Local Outlier Factor Anomaly Detection'),
 Model('Anomaly Detection with Supervised Learning (XGB)')]

In [28]:
# get anomaly detection predictions
print(str(datetime.now()))
preds_ad = prep_ad_preds(models)
print(str(datetime.now()))

2020-04-11 17:44:15.539194
2020-04-11 17:46:44.567550


In [29]:
# read the origonal dataset and add anomaly detection predictions
df = pd.read_csv('data/DR_Demo_Medical_Fraud.csv')
print(df.shape)

df = df.merge(preds_ad, left_index=True, right_index=True)
print(df.shape)

(12243, 17)
(12243, 21)


In [30]:
# create project with additional features and the same partitioning
project_clf_ad = dr.Project.create(project_name=f'{project_name}_clf_ad', sourcedata=df)

part = dr.UserCV(user_partition_col='partition_id', 
                 cv_holdout_level=5)
project_clf_ad.set_target(target=target,
                          mode=dr.enums.AUTOPILOT_MODE.FULL_AUTO,
                          metric=metric,
                          worker_count=-1, 
                          partitioning_method=part)

Project(Medical_Insurance_Fraud_clf_ad)

In [ ]:
project_clf_ad.wait_for_autopilot()

In [32]:
# get the leaderboard
model_scores_ad = get_model_scores(project_clf_ad, metric='AUC', ascending=False)

In [33]:
model_scores_ad.head()

,model_number,model_type,model_category,model,sample_pct,metric_v,metric_cv
0,125,ENET Blender,blend,Model('ENET Blender'),64.00393,0.78126,0.775316
5,123,AVG Blender,blend,Model('AVG Blender'),64.00393,0.77029,0.770706
4,126,ENET Blender,blend,Model('ENET Blender'),64.00393,0.77036,0.770556
10,73,Light Gradient Boosting on ElasticNet Predicti...,model,Model('Light Gradient Boosting on ElasticNet P...,64.00393,0.76354,0.767978
1,124,Advanced AVG Blender,blend,Model('Advanced AVG Blender'),64.00393,0.77917,0.766440


### 3. Compare results

In [36]:
model_clf = dr.ModelRecommendation.get(project_clf.id).get_model()
model_clf_ad = dr.ModelRecommendation.get(project_clf_ad.id).get_model()

In [37]:
# recommended models results
print('AUC supervised                      :', model_clf.metrics['AUC']['crossValidation'])
print('AUC supervised with anomaly features:', model_clf_ad.metrics['AUC']['crossValidation'])

print('LogLoss supervised                      :', model_clf.metrics['LogLoss']['crossValidation'])
print('LogLoss supervised with anomaly features:', model_clf_ad.metrics['LogLoss']['crossValidation'])

AUC supervised                      : 0.77278
AUC supervised with anomaly features: 0.765288
LogLoss supervised                      : 0.16669399999999998
LogLoss supervised with anomaly features: 0.167132


### 4. Deployment and scoring

In [ ]:
def predict_deployment(data, datarobot_key, deployment_url, deployment_id):
    # Set HTTP headers. The charset should match the contents of the file.
    headers = {'Content-Type': 'application/json; charset=UTF-8', 'datarobot-key': datarobot_key}

    url = f'{deployment_url}/predApi/v1.0/deployments/{deployment_id}/predictions'
    
    # Make API request for predictions
    predictions_response = requests.post(
        url,
        auth=(creds['username'], creds['token']),
        data=data,
        headers=headers,
    )

    return predictions_response.json()

In [38]:
def predict_deployment_expl(data, datarobot_key, deployment_url, deployment_id):
    # Set HTTP headers. The charset should match the contents of the file.
    headers = {'Content-Type': 'application/json; charset=UTF-8', 'datarobot-key': datarobot_key}
    
    # prediction explanations parameters
    params = {
            'maxCodes': 3,
            'thresholdHigh': 0.1,
            'thresholdLow': 0.01,
        }
    
    url = f'{deployment_url}/predApi/v1.0/deployments/{deployment_id}/predictionExplanations'
    
    # Make API request for predictions
    predictions_response = requests.post(
        url,
        auth=(creds['username'], creds['token']),
        data=data,
        headers=headers,
        params=params
    )

    return predictions_response.json()

In [39]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [40]:
# compute feature impact
feature_impacts = model_clf.get_or_request_feature_impact()

# initialize prediction explanations
pei_job = dr.PredictionExplanationsInitialization.create(project_clf.id, model_clf.id)
pei_job.wait_for_completion()

In [41]:
# deploy the model
deployment = dr.Deployment.create_from_learning_model(model_id=model_clf.id, 
                                                      label=f'{project_name}_clf_depl',
                                                      default_prediction_server_id=creds['pred_serv_id'])

In [42]:
# get prediction server url, deployment id and DataRobot key
pred_server = deployment.default_prediction_server

dr_key = pred_server['datarobot-key']
depl_url = pred_server['url']
depl_id = deployment.id

In [43]:
# read and prepare a dataset to score
df_scoring = pd.read_csv('data/DR_Demo_Medical_Fraud_scoring.csv')
print(df_scoring.shape)
data_to_pred = json.dumps(df_scoring.to_dict(orient='records'))

(20000, 18)


In [44]:
# get and precess predictions with explanations
print(str(datetime.now()))
preds_raw = predict_deployment_expl(data_to_pred, dr_key, depl_url, depl_id)

df_preds = pd.DataFrame()
for row in preds_raw['data']:
    new_row = pd.json_normalize(data=flatten_json(row))
    df_preds = pd.concat([df_preds, new_row])
print(str(datetime.now()))

2020-04-11 18:55:09.722345
2020-04-11 19:02:09.550274


In [45]:
cols_to_rename = {'predictionValues_0_value': 'Prediction', 
                  'predictionExplanations_0_feature': 'Primary Feature',
                  'predictionExplanations_0_featureValue': 'Primary Feature Value',
                  'predictionExplanations_0_qualitativeStrength':'Primary Feature Strength',
                  
                  'predictionExplanations_1_feature': 'Secondary Feature',
                  'predictionExplanations_1_featureValue': 'Secondary Feature Value',
                  'predictionExplanations_1_qualitativeStrength':'Secondary Feature Strength',
                  
                  'predictionExplanations_2_feature': 'Tertiary Feature',
                  'predictionExplanations_2_featureValue': 'Tertiary Feature Value',
                  'predictionExplanations_2_qualitativeStrength':'Tertiary Feature Strength'}

df_preds.rename(columns=cols_to_rename, inplace=True)

In [46]:
print(df_scoring.shape, df_preds.shape)
cols_to_add = ['rowId', 'Prediction', 
               'Primary Feature Value', 'Primary Feature Strength', 'Primary Feature', 
               'Secondary Feature Value', 'Secondary Feature Strength', 'Secondary Feature',
               'Tertiary Feature Value', 'Tertiary Feature Strength', 'Tertiary Feature',
              ]
df_scoring = df_scoring.merge(df_preds[cols_to_add], on='rowId')
df_scoring['Prediction Category'] = df_scoring.Prediction.apply(lambda x:
                                                                'High' if x >= 0.2 else
                                                                'Medium' if x >= 0.1 else 'Low')
print(df_scoring.shape)

(20000, 18) (20000, 22)
(20000, 29)


In [47]:
# save results to use for a dashboard preparation
df_scoring.to_csv('data/DR_Demo_Medical_Fraud_scoring_predictions.csv', index=False)